<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 3</font>
## <font color='blue'>Fine-Tuning de LLM Open-Source Para App de Assistente Jurídico</font>

## Instalando e Carregando Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00


In [2]:
# Instalando os pacotes
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00


In [3]:
# Imports
import numpy as np
import nltk
import shutil
import evaluate
import transformers
import datasets
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [5]:
# Remove as pastas, se já existirem. Isso evita erros ao executar o jupyter a partir da segunda vez em diante
try:
    shutil.rmtree('logs_treino')
    shutil.rmtree('resultados_treino')
    shutil.rmtree('modelo_salvo')
except:
    print('As pastas não existem ou já foram removidas!')

As pastas não existem ou já foram removidas!


## Carregando os Dados

In [6]:
# Define o nome do arquivo
nome_arquivo = "dataset.csv"

In [7]:
# Carrega os dados
dsa_dataset = load_dataset('csv', data_files = nome_arquivo, delimiter = ',')

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
# Divisão em treino e teste com proporção 80/20
dsa_dataset = dsa_dataset["train"].train_test_split(test_size = 0.2)

In [9]:
# Visualiza o formato do dataset
dsa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 749
    })
})

## Tokenizador e LLM Open-Source

https://huggingface.co/google/flan-t5-base

In [10]:
# Carrega o tokenizador
tokenizador = T5Tokenizer.from_pretrained("google/flan-t5-base", legacy = False)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [11]:
# Visualiza o tokenizador
tokenizador

T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '

In [12]:
# Carrega o LLM pré-treinado
modelo = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
# Visualiza a arquitetura do LLM
modelo

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [14]:
# Data collator para concatenar tokenizador e modelo
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizador, model = modelo)

In [15]:
# Visualiza o data collator
data_collator

DataCollatorForSeq2Seq(tokenizer=T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<

## Pré-Processamento

In [16]:
# Todos os inputs vão receber como prefixo: "answer the question" (responda a questão)
prefix = "answer the question: "

In [17]:
# Função de pré-processamento
def dsa_fn_preprocess(data):

    # Concatena o prefixo a cada pergunta na lista de perguntas fornecida em data["question"]
    inputs = [prefix + doc for doc in data["question"]]

    # Usa o tokenizer para converter as perguntas processadas em tokens com um comprimento máximo de 128,
    # truncando as que forem mais longas
    model_inputs = tokenizador(inputs, max_length = 128, truncation = True)

    # Tokeniza as respostas fornecidas em data["answer"] com um comprimento máximo de 512,
    # truncando as que forem mais longas
    labels = tokenizador(text_target = data["answer"], max_length = 512, truncation = True)

    # Adiciona os tokens das respostas como rótulos no dicionário de entradas do modelo
    model_inputs["labels"] = labels["input_ids"]

    # Retorna o dicionário processado contendo tanto as entradas tokenizadas quanto os rótulos
    return model_inputs

In [18]:
# Aplica a função de pré-processamento ao dataset gerando o dataset tokenizado
dsa_dataset_tokenized = dsa_dataset.map(dsa_fn_preprocess, batched = True)

Map:   0%|          | 0/2993 [00:00<?, ? examples/s]

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

In [19]:
# Visualiza o dataset tokenizado
dsa_dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 749
    })
})

In [20]:
dsa_dataset_tokenized['train']['question'][0]

'Q: Hello I had somebody give me money to start a business that never got off the ground.. Business never got into full swing. Problems on top of problems and no more money to continue forward. It never got off the ground, am I required to give the money back ? I never signed anything and only talked about hoping it would be successful. It never made it to an official business and it was hopes for to make it so they could get a double return. '

In [21]:
dsa_dataset_tokenized['train']['answer'][0]

'A:Hi, keep in mind that in addition to written agreements, there are also verbal agreements that may be legally enforceable, provided there is an offer, an acceptance, and consideration. Whether you are required to return the funds depends on what were the terms of the agreement. The other person may be entitled to a partial refund or full refund to prevent unjust enrichment or to recover legal damages. There is not enough information here to determine whether there is a cause of action. When there is no written agreement, it is difficult to litigate these cases because the parties will resort to preliminary conversations to determine what were the expectations of the parties, and whether they were fulfilled. You may want to talk to the investor and see whether he/she understands that the business did not take off for reasons outside of your control and whether he/she is expecting any recovery. After that conversation, you will have a better idea of whether you need to retain an attor

In [22]:
dsa_dataset_tokenized['train']['input_ids'][0]

[1525,
 8,
 822,
 10,
 1593,
 10,
 8774,
 27,
 141,
 10843,
 428,
 140,
 540,
 12,
 456,
 3,
 9,
 268,
 24,
 470,
 530,
 326,
 8,
 1591,
 5,
 5,
 1769,
 470,
 530,
 139,
 423,
 7180,
 5,
 5289,
 7,
 30,
 420,
 13,
 982,
 11,
 150,
 72,
 540,
 12,
 916,
 1039,
 5,
 94,
 470,
 530,
 326,
 8,
 1591,
 6,
 183,
 27,
 831,
 12,
 428,
 8,
 540,
 223,
 3,
 58,
 27,
 470,
 3814,
 959,
 11,
 163,
 6812,
 81,
 6055,
 34,
 133,
 36,
 1574,
 5,
 94,
 470,
 263,
 34,
 12,
 46,
 2314,
 268,
 11,
 34,
 47,
 7511,
 21,
 12,
 143,
 34,
 78,
 79,
 228,
 129,
 3,
 9,
 1486,
 1205,
 5,
 1]

In [23]:
dsa_dataset_tokenized['train']['labels'][0]

[71,
 10,
 12146,
 6,
 453,
 16,
 809,
 24,
 16,
 811,
 12,
 1545,
 10663,
 6,
 132,
 33,
 92,
 7375,
 138,
 10663,
 24,
 164,
 36,
 13595,
 3,
 31814,
 6,
 937,
 132,
 19,
 46,
 462,
 6,
 46,
 11122,
 6,
 11,
 4587,
 5,
 3,
 2374,
 25,
 33,
 831,
 12,
 1205,
 8,
 2731,
 5619,
 30,
 125,
 130,
 8,
 1353,
 13,
 8,
 2791,
 5,
 37,
 119,
 568,
 164,
 36,
 7201,
 12,
 3,
 9,
 11807,
 7147,
 42,
 423,
 7147,
 12,
 1709,
 73,
 4998,
 12812,
 297,
 42,
 12,
 8303,
 1281,
 8542,
 5,
 290,
 19,
 59,
 631,
 251,
 270,
 12,
 2082,
 823,
 132,
 19,
 3,
 9,
 1137,
 13,
 1041,
 5,
 366,
 132,
 19,
 150,
 1545,
 2791,
 6,
 34,
 19,
 1256,
 12,
 22427,
 342,
 175,
 1488,
 250,
 8,
 2251,
 56,
 4631,
 12,
 17413,
 9029,
 12,
 2082,
 125,
 130,
 8,
 4454,
 13,
 8,
 2251,
 6,
 11,
 823,
 79,
 130,
 20795,
 5,
 148,
 164,
 241,
 12,
 1350,
 12,
 8,
 12024,
 11,
 217,
 823,
 3,
 88,
 87,
 7,
 88,
 734,
 7,
 24,
 8,
 268,
 410,
 59,
 240,
 326,
 21,
 2081,
 1067,
 13,
 39,
 610,
 11,
 823,
 3,
 88,
 87,
 7,

## Definindo a Métrica de Avaliação do Modelo

In [24]:
# O pacote "punkt" é específico para a tarefa de tokenização, que envolve a divisão de um texto
# em uma lista de sentenças
nltk.download("punkt", quiet = True)

True

In [25]:
# Download do módulo complementar punkt_tab
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [26]:
# Define a métrica
metric = evaluate.load("rouge")

In [27]:
# Função de cálculo das métricas
def dsa_calcula_metricas(eval_preds):

    # Desempacota as predições e os rótulos do argumento eval_preds
    previsoes, labels = eval_preds

    # Substitui todos os valores diferentes de -100 em labels pelo ID do token de preenchimento
    labels = np.where(labels != -100,
                      labels,
                      tokenizador.pad_token_id)

    # Decodifica as previsões para texto, ignorando tokens especiais
    previsoes_decodificadas = tokenizador.batch_decode(previsoes,
                                                       skip_special_tokens = True)

    # Decodifica os labels para texto, ignorando tokens especiais
    labels_decodificados = tokenizador.batch_decode(labels,
                                                    skip_special_tokens = True)

    # Adiciona uma nova linha após cada sentença para as previsões decodificadas,
    # preparando-as para a avaliação ROUGE
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in previsoes_decodificadas]

    # Adiciona uma nova linha após cada label para as previsões decodificadas,
    # preparando-os para a avaliação ROUGE
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in labels_decodificados]

    # Calcula a métrica ROUGE entre as previções e os rótulos decodificados, utilizando um stemmer
    resultado = metric.compute(predictions = previsoes_decodificadas,
                               references = labels_decodificados,
                               use_stemmer = True)

    # Retorna o resultado da métrica ROUGE
    return resultado

In [28]:
# Define argumentos de treino
dsa_training_args = Seq2SeqTrainingArguments(output_dir = "resultados_treino",
                                             eval_strategy = "epoch",
                                             learning_rate = 3e-4,
                                             logging_dir = "logs_treino",
                                             logging_steps = 1,
                                             per_device_train_batch_size = 4,
                                             per_device_eval_batch_size = 2,
                                             weight_decay = 0.01,
                                             save_total_limit = 3,
                                             num_train_epochs = 3,
                                             predict_with_generate = True,
                                             push_to_hub = False,
                                             report_to = "none")

In [29]:
# Define o trainer
dsa_trainer = Seq2SeqTrainer(model = modelo,
                             args = dsa_training_args,
                             train_dataset = dsa_dataset_tokenized["train"],
                             eval_dataset = dsa_dataset_tokenized["test"],
                             tokenizer = tokenizador,
                             data_collator = data_collator,
                             compute_metrics = dsa_calcula_metricas)

## Treinamento do Modelo

In [30]:
%%time
dsa_trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.186200,2.456064,0.134931,0.030167,0.106872,0.106834
2,3.098900,2.413667,0.133134,0.031292,0.106453,0.106233
3,1.782000,2.411009,0.132457,0.032308,0.106658,0.106557


CPU times: user 17min 7s, sys: 34.6 s, total: 17min 42s
Wall time: 17min 56s


TrainOutput(global_step=2247, training_loss=2.4596848632157196, metrics={'train_runtime': 1075.7091, 'train_samples_per_second': 8.347, 'train_steps_per_second': 2.089, 'total_flos': 1469863755030528.0, 'train_loss': 2.4596848632157196, 'epoch': 3.0})

In [31]:
# Salva o modelo
dsa_trainer.save_model('modelo_salvo')

**ROUGE-1** mede a sobreposição de unigramas (palavras individuais).

**ROUGE-2** mede a sobreposição de bigramas (pares de palavras consecutivas).

**ROUGE-L** mede a sobreposição da subsequência mais longa comum entre o resumo gerado e o resumo de referência. Isso leva em conta a ordem das palavras, mas permite lacunas. Valores mais altos indicam melhor desempenho. ROUGE-L é calculado com base na similaridade entre as sequências, levando em consideração a precisão, o recall e a média harmônica entre eles.

Valores mais altos de ROUGE indicam uma maior semelhança entre o resumo gerado e o resumo de referência, o que geralmente é interpretado como uma indicação de melhor qualidade do resumo. No entanto, é importante lembrar que nenhuma métrica pode capturar completamente a qualidade de um resumo e é útil complementar a avaliação com análises qualitativas ou outras métricas.

## Deploy e Uso do Modelo

In [32]:
# Carrega o tokenizador final salvo em disco
tokenizador_dsa_final = AutoTokenizer.from_pretrained('modelo_salvo')

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [33]:
# Carrega o modelo final salvo em disco
modelo_dsa_final = AutoModelForSeq2SeqLM.from_pretrained('modelo_salvo')

In [34]:
# Visualiza a arquitetura do modelo
modelo_dsa_final

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [35]:
# Texto de entrada
texto_entrada = "Can I move out of state with my children if I have a custody agreement in that state?"

In [36]:
# Tokeniza o input
texto_entrada_tokenizado = tokenizador_dsa_final(texto_entrada, return_tensors = "pt").input_ids

In [37]:
# Gera a saída (previsão do modelo)
texto_saida_tokenizado = modelo_dsa_final.generate(texto_entrada_tokenizado,
                                                   max_length = 50,
                                                   temperature = 0.4,
                                                   do_sample = True)

In [38]:
# Visualiza
texto_saida_tokenizado

tensor([[    0,    71, 16701,  2791,    16,    24,   538,    54,    36, 13595,
         11293,     6,    68,    34,    31,     7,   359,    12,  2232,    24,
            34,    31,     7,    59, 11293,    30,    25,    12,   888,    91,
            13,   538,     5,   156,    25,    43,     3,     9, 16701,  2791,
            16,    24,   538,     6,    25,   164,    43,     8,   269,    12]])

In [39]:
# Decode da saída
texto_saida = tokenizador_dsa_final.decode(texto_saida_tokenizado[0],
                                           skip_special_tokens = True)

In [40]:
# Resultado
print("Pergunta:", texto_entrada)
print("Resposta:", texto_saida)

Pergunta: Can I move out of state with my children if I have a custody agreement in that state?
Resposta: A custody agreement in that state can be legally binding, but it's important to note that it's not binding on you to move out of state. If you have a custody agreement in that state, you may have the right to


In [41]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [42]:
#%watermark -v -m

In [43]:
#%watermark --iversions

# Fim